# Active Directory Most Vulnerable Path Identification

## Imports & globals

### Imports

In [5]:
import pandas as pd
import networkx as nx
import numpy as np
from py2neo import Graph
import random
import torch
from torch_geometric.data import Data, HeteroData
import json
import os
from torch_geometric.nn import to_hetero, GAT, SAGEConv
from torch_geometric.explain import Explanation
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


### Global variables

In [153]:
# Database connection details
graph = Graph("http://localhost:7474", auth=("neo4j", "bloodhoundcommunityedition"))

# Object types and their corresponding properties
object_types_and_properties = {
    'Domain': ['name', 'objectid', 'highvalue'],
    'User': ['name', 'objectid', 'admincount', 'dontreqpreauth', 'pwdneverexpires', 'hasspn', 'highvalue', 'savedcredentials',
             'passwordnotreqd', 'pwdlastset', 'lastlogon', 'unconstraineddelegation', 'enabled', 'sensitive'],
    'Computer': ['name', 'objectid', 'operatingsystem', 'enabled', 'haslaps', 'highvalue', 'lastlogontimestamp', 
                 'pwdlastset', 'unconstraineddelegation', 'privesc', 'creddump', 
                 'exploitable'],
    'Group': ['name', 'objectid', 'highvalue', 'admincount'],
    'OU': ['name', 'objectid', 'highvalue', 'blocksInheritance'],
    'GPO': ['name', 'objectid', 'exploitable'],
    'Container': ['name', 'objectid', 'highvalue']
}

# Relationship types
relationship_types = [
    'AddMember',
    'AddSelf',
    'AdminTo',
    'AllExtendedRights',
    'AllowedToAct',
    'AllowedToDelegate',
    'CanPSRemote',
    'CanRDP',
    'Contains',
    'ExecuteDCOM',
    'ForceChangePassword',
    'GenericAll',
    'GenericWrite',
    'GetChanges',
    'GetChangesAll',
    'GpLink',
    'HasSession',
    'MemberOf',
    'Owns',
    'ReadLAPSPassword',
    'SQLAdmin',
    'WriteDacl',
    'WriteOwner'
]

# OS possibilities
global_os_categories = ['Windows Server 2003 Enterprise Edition', 'Windows Server 2008 Datacenter', 'Windows Server 2008 Enterprise', 
                        'Windows Server 2008 R2 Datacenter', 'Windows Server 2008 R2 Enterprise', 'Windows Server 2008 R2 Standard', 
                        'Windows Server 2008 Standard', 'Windows Server 2012 Datacenter', 'Windows Server 2012 R2 Datacenter', 
                        'Windows Server 2012 R2 Standard', 'Windows Server 2012 Standard', 'Windows Server 2016 Datacenter', 
                        'Windows Server 2016 Standard']

# Object property types
object_property_types = {
    "Domain": {
        "Name": "string",
        "Objectid": "string",
        "Highvalue": "boolean"
    },
    "User": {
        "Name": "string",
        "Objectid": "string",
        "Admincount": "boolean",
        "Dontreqpreauth": "boolean",
        "Pwdneverexpires": "boolean",
        "Hasspn": "boolean",
        "Highvalue": "boolean",
        "Savedcredentials": "boolean",
        "Passwordnotreqd": "boolean",
        "Pwdlastset": "numerical",
        "Lastlogon": "numerical",
        "Unconstraineddelegation": "boolean",
        "Enabled": "boolean",
        "Sensitive": "boolean"
    },
    "Computer": {
        "Name": "string",
        "Objectid": "string",
        "Operatingsystem": "categorical",
        "Enabled": "boolean",
        "Haslaps": "boolean",
        "Highvalue": "boolean",
        "Lastlogontimestamp": "numerical",
        "Pwdlastset": "numerical",
        "Unconstraineddelegation": "boolean",
        "Privesc": "boolean",
        "Creddump": "boolean",
        "Exploitable": "boolean"
    },
    "Group": {
        "Name": "string",
        "Objectid": "string",
        "Highvalue": "boolean",
        "Admincount": "boolean"
    },
    "OU": {
        "Name": "string",
        "Objectid": "string",
        "Highvalue": "boolean",
        "Blocksinheritance": "boolean"
    },
    "GPO": {
        "Name": "string",
        "Objectid": "string",
        "Exploitable": "boolean"
    },
    "Container": {
        "Name": "string",
        "Objectid": "string",
        "Highvalue": "boolean"
    }
}

# Torch print options
torch.set_printoptions(threshold=float('inf'), linewidth=1000)
#torch.set_printoptions(profile="default")

## Functions for handling graph database

In [7]:
def clear_neo4j_database(session):
    # Delete nodes and edges with batching into 10k objects - From DBCreator
    total = 1
    while total > 0:
        result = session.run(
            "MATCH (n) WITH n LIMIT 10000 DETACH DELETE n RETURN count(n)")
        for r in result:
            total = int(r['count(n)'])
    session.run("CALL apoc.schema.assert({},{},true);")
    
        # Remove constraint - From DBCreator
    for constraint in session.run("SHOW CONSTRAINTS"):
        session.run("DROP CONSTRAINT {}".format(constraint['name']))

    icount = session.run(
        "SHOW INDEXES YIELD name RETURN count(*)")
    for r in icount:
        ic = int(r['count(*)'])
            
    while ic >0:
    
        showall = session.run(
            "SHOW INDEXES")
        for record in showall:
            name = (record['name'])
            session.run("DROP INDEX {}".format(name))
        ic = 0
        
    # Setting constraints
    constraints = [
            "CREATE CONSTRAINT FOR (n:Base) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:Domain) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:Computer) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:User) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:OU) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:GPO) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:Compromised) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:Group) REQUIRE n.neo4jImportId IS UNIQUE;",
            "CREATE CONSTRAINT FOR (n:Container) REQUIRE n.neo4jImportId IS UNIQUE;",
    ]

    for constraint in constraints:
        try:
            session.run(constraint)
        except:
            continue
    
    session.run("match (a) -[r] -> () delete a, r")
    session.run("match (a) delete a")

In [8]:
def load_graph_from_json(session, file_path):
    """
    Loads a graph from a JSON file into Neo4j.
    """
    with open(file_path, 'r') as f:
        query = f"PROFILE CALL apoc.periodic.iterate(\"CALL apoc.import.json('{file_path}')\", \"RETURN 1\", {{batchSize:1000}})"
        session.run(query)

## Functions for extracting features and creating dataset

#### Extract features from neo4j database

In [9]:
# Function to extract features from the Neo4j database for a specific object type and returns a Pandas DataFrame.
def extract_features(graph, labels, properties):

    # Create the RETURN clause dynamically based on the provided properties
    return_clause = ", ".join([f"n.{prop} AS node_{prop}" for prop in properties])

    # Define the Cypher query with labels and properties
    query = f"""
    MATCH (n:{labels})
    RETURN 
        id(n) AS node_id, 
        {return_clause}
    """

    # Execute the query and store the results in a Pandas DataFrame
    result = graph.run(query)
    df = pd.DataFrame(result)

    if df.empty:  # Check if the DataFrame is empty
        df = pd.DataFrame(columns=['Node ID'] + [prop.title() for prop in properties]) 

    # Add headers to the DataFrame (adjust based on properties)
    df.columns = ['Node ID'] + [prop.title() for prop in properties]

    return df

In [10]:
def get_node_properties(graph, label):
    query = f"""
    MATCH (n:{label})
    WITH keys(n) AS keys
    UNWIND keys AS key
    RETURN DISTINCT key
    """
    result = graph.run(query)
    return [record["key"] for record in result]

all_possible_object_types_and_properties = {
    label: get_node_properties(graph, label) 
    for label in ['Domain', 'User', 'Computer', 'Group', 'GPO', 'Container']
}

In [11]:
# Function to extract relationships from the Neo4j database and returns a Pandas DataFrame.
def extract_relationships(graph, rel_types):
    
    # List to store DataFrames for each relationship type
    dfs = []

    for rel_type in rel_types:
        # Define the Cypher query with dynamic relationship type
        query = f"""
        MATCH (source)-[r:{rel_type}]->(target)
        RETURN 
            id(source) AS source_id,
            id(target) AS target_id,
            TYPE(r) AS relationship_type
        """

        # Execute the query and store the results in a Pandas DataFrame
        result = graph.run(query)
        df = pd.DataFrame(result)
        
        if not df.empty:

            # Add headers to the DataFrame
            df.columns = ['Source ID', 'Target ID', 'Relationship Type']

            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    return pd.concat(dfs, ignore_index=True)

#### Filter dataframes

In [12]:
def filter_dataframes(dfs):
    for object_name in dfs:
        for property in dfs[object_name].columns:
            # Take first element if it is a list
            dfs[object_name][property] = dfs[object_name][property].apply(lambda x: x[0] if isinstance(x, list) else x)
            # Set boolean to False if null
            dfs[object_name][property] = dfs[object_name][property].apply(lambda x: False if x == "null" else x)
    return dfs

#### Add columns

In [13]:
def add_columns (object_dfs):
    for df_name in object_dfs:
        # Add score column
        object_dfs[df_name]['Score'] = 0

        # Add source/target node column
        object_dfs[df_name]['Source/Target'] = 0

        #Add shortest path coumn
        object_dfs[df_name]['Shortest Path'] = 0
    return object_dfs

## Calculate scores 

### Nodes

In [14]:
def add_columns (object_dfs):
    for df_name in object_dfs:
        # Add score column
        object_dfs[df_name]['Score'] = 0

        # Add source/target node column
        object_dfs[df_name]['Source/Target'] = 0

        #Add shortest path coumn
        object_dfs[df_name]['Shortest Path'] = 0
    return object_dfs

In [15]:
# Convert features to scores
# For now generated by Guiseppe paper/own insight, assign more precise scores later

features = {}

# Domain features
features['Domain'] = {
    "Highvalue": {True: 10, False: 0} # A compromised domain is extremely high value.
}

# Computer features
features['Computer'] = {
    "Operatingsystem": {
        "Windows Server 2003 Enterprise Edition": 29,
        "Windows XP Professional Service Pack 3": 29,
        "Windows Server 2008 Standard": 27,
        "Windows Server 2008 Datacenter": 27,
        "Windows Server 2008 Enterprise": 27,
        "Windows 7 Professional Service Pack 1": 22,
        "Windows 7 Ultimate Service Pack 1": 22,
        "Windows 7 Enterprise Service Pack 1": 22,
        "Windows Server 2008 R2 Standard": 20,
        "Windows Server 2008 R2 Datacenter": 20,
        "Windows Server 2008 R2 Enterprise": 20,
        "Windows Server 2012 Standard": 17,
        "Windows Server 2012 Datacenter": 17,
        "Windows Server 2012 R2 Standard": 14, 
        "Windows Server 2012 R2 Datacenter": 14,
        "Windows Server 2016 Standard": 10,
        "Windows Server 2016 Datacenter": 10,
        "Windows 10 Pro": 7,
        "Windows 10 Enterprise": 3,
    },
    "Enabled": {True: 1, False: 0},
    "Haslaps": {True: 0, False: 1},
    "Highvalue": {True: 5, False: 0},
    "Lastlogontimestamp": lambda x: 1 if x == 0.0 else 0,  # Higher score if never logged on
    "Pwdlastset": lambda x: 1 if x == 0.0 or x == -1.0 else 0,  # Higher score if never set or unknown
    "Unconstraineddelegation": {True: 5, False: 0},
    "Privesc": {True: 10, False: 0},
    "Creddump": {True: 7, False: 0},
    "Exploitable": {True: 10, False: 0},
}

# GPO features
features['GPO'] = {
    "Exploitable": {True: 10, False: 0}
}

# Group features
features['Group'] = {
    "Highvalue": {True: 10, False: 0},
    "Admincount": {True: 10, False: 0}
}

# OU features
features['OU'] = {
    "Highvalue": {True: 10, False: 0},
    "Blocksinheritance": {True: 10, False: 0}
}


# User features
features['User'] = {
    "Admincount": {True: 5, False: 0},
    "Dontreqpreauth": {True: 5, False: 0},
    "Pwdneverexpires": {True: 3, False: 0},
    "Hasspn": {True: 5, False: 0},
    "Highvalue": {True: 5, False: 0},
    "Savedcredentials": {True: 3, False: 0},
    "Passwordnotreqd": {True: 10, False: 0},
    "Pwdlastset": lambda x: 3 if x == 0.0 or x == -1.0 else 0,  # Higher score if never set or unknown
    "Lastlogon": lambda x: 1 if x == 0.0 or x == -1.0 else 0,  # Higher score if never logged on or unknown
    "Unconstraineddelegation": {True: 5, False: 0},
    "Enabled": {True: 1, False: 0},
    "Sensitive": {True: 3, False: 0},
}

# Container features
features['Container'] = {
    "Highvalue": {True: 5, False: 0} # Containers are less critical than domains or OUs but still can hold sensitive objects.
}


In [16]:
#Calculate and add scores for each of the objects
def calculate_and_add_scores(object_dfs):
    # For every dataframe
    for df_name in object_dfs:
        # For every row (node) in dataframe
        for i in object_dfs[df_name].index:
            score = 0
            # For every feature of the node (except name and id)
            for feature, values in features[df_name].items():
                feature_value = object_dfs[df_name].loc[i, feature]

                # Check if lambda function
                if callable(values):
                    score += values(feature_value)
                # Check if dictionary
                elif isinstance(values, dict):
                    # Check if list
                    if isinstance(feature_value, list):
                        for item in feature_value:
                            if item in values:
                                score += values[item]
                    # Check if normal value
                    elif feature_value in values:
                        score += values[feature_value]

            # Assign score
            object_dfs[df_name].loc[i, 'Score'] = score
    return object_dfs


### Relationships

In [17]:
relationship_scores = {
    "Computer": {
        "MemberOf": 15,
        "AdminTo": 2,
        "HasSession": 2,
        "CanRDP": 2,
        "CanPSRemote": 2,
        "ExecuteDCOM": 2,
        "AllowedToAct": 5,
        "AllowedToDelegate": 5,
        "Contains": 8,
        "SQLAdmin": 10,
        "WriteDacl": 10,        
        "WriteOwner": 10,       
    },
    "Domain": {
        "TrustedBy": 3,
        "Contains": 20,
        "AddSelf": 8    
    },
    "GPO": {
        "Contains": 8,
        "GPLink": 4      
    },
    "Group": {
        "MemberOf": 30,
        "AdminTo": 10,
        "CanRDP": 2,
        "CanPSRemote": 2,
        "ExecuteDCOM": 2,
        "Contains": 20,
        "AddMember": 8, 
        "GenericAll": 10,   
    },
    "OU": {
        "Contains": 15
    },
    "User": {
        "MemberOf": 35,
        "AdminTo": 10,
        "HasSession": 3,
        "CanRDP": 5,
        "CanPSRemote": 5,
        "ExecuteDCOM": 5,
        "AllowedToAct": 5,
        "AllowedToDelegate": 5, 
        "Contains": 20,
        "ReadLAPSPassword": 10,
        "ALLExtendedRights": 8,
        "GenericWrite": 8,
        "Owns": 10, 
        "ForceChangePassword": 8,
        "GetChangesAll": 6,
        "GetChanges": 6          
    }
}

In [18]:
# Function to get relationship score based on the table
def get_relationship_score(source_type, relationship_type):
    if source_type in relationship_scores and relationship_type in relationship_scores[source_type]:
        score = relationship_scores[source_type][relationship_type]
        return score
    else:
        # print(f"Can't find {relationship_type}")
        return 1

In [19]:
# Add relationship scores to node scores
def add_relationship_scores(object_dfs, df_rels):
    for df_name in object_dfs:
        for i in object_dfs[df_name].index:
            source_id = object_dfs[df_name].loc[i, 'Node ID']
            outgoing_relationships = df_rels[df_rels['Source ID'] == source_id]

            for _, row in outgoing_relationships.iterrows():
                relationship_type = row['Relationship Type']
                relationship_score = get_relationship_score(df_name, relationship_type)
                object_dfs[df_name].loc[i, 'Score'] += relationship_score
    return object_dfs

## Calculating shortest path

In [20]:
# Function to map name to node id
def get_node_name(object_dfs, node_id, dataframes):
  # Iterate through each DataFrame
  for df_name in dataframes:
    # Look for node id
    node = object_dfs[df_name][object_dfs[df_name]['Node ID'] == node_id]
    if not node.empty:
      # If found, return the corresponding name
      return node['Name'].iloc[0]  
  # If not found in any DataFrame, return None
  return None

In [21]:
# Create node and relationship dataframes
def create_dataframes(object_dfs, df_rels):
    # Initialize empty list
    extracted_data = []

    # Iterate through each DataFrame
    for df_name in object_dfs:
        # Extract the 'Node ID' and 'Score' columns
        extracted_df = object_dfs[df_name][['Node ID', 'Score', 'Source/Target', 'Shortest Path']].copy()
        extracted_data.append(extracted_df)

    # Concatenate all extracted DataFrames into a single DataFrame
    combined_df = pd.concat(extracted_data, ignore_index=True)

    # Get all unique Node IDs from combined_df
    valid_nodes = combined_df['Node ID']

    # Filter df_rels to keep only relationships between valid nodes
    df_rels_filtered = df_rels[
        df_rels['Source ID'].isin(valid_nodes) & df_rels['Target ID'].isin(valid_nodes)
    ].copy()
    return combined_df, df_rels_filtered

In [22]:
def calculate_shortest_path(combined_df, df_rels_filtered, object_dfs):
# HIER GEBLEVEN!
  # Create NetworkX graph
  nx_graph = nx.DiGraph()

  # Add nodes with 'Score' as weight
  for _, row in combined_df.iterrows():
      nx_graph.add_node(row['Node ID'], weight=1/(row['Score']+1e-10))

  # Add edges
  for _, row in df_rels_filtered.iterrows():
      nx_graph.add_edge(row['Source ID'], row['Target ID'])

  shortest_path_found = False
  while(True):
    # Select random user node
    source_node = int(object_dfs['User'][object_dfs['User']['Objectid'].str.endswith(str(random.randint(10,99)))]['Node ID'].iloc[0])
    # Select domain admin node
    target_node = int(object_dfs['Group'][object_dfs['Group']['Objectid'].str.endswith('-512')]['Node ID'].iloc[0])

    # Run Dijkstra
    try:
        shortest_path = nx.dijkstra_path(nx_graph, source=source_node, target=target_node, weight='weight')
        break
    except nx.NetworkXNoPath:
        print(f"No path found between {source_node} and {target_node}")
  return source_node, target_node, shortest_path


In [23]:
# Add shortest path to node features

# For every dataframe
def add_shortest_path(object_dfs, shortest_path, target_node, source_node):
    for df_name in object_dfs:
        # For every row (node) in dataframe
        for i in object_dfs[df_name].index:
            if object_dfs[df_name].loc[i, 'Node ID'] in shortest_path:
                object_dfs[df_name].loc[i, 'Shortest Path'] = 1
            if object_dfs[df_name].loc[i, 'Node ID'] == source_node or object_dfs[df_name].loc[i, 'Node ID'] == target_node:
                object_dfs[df_name].loc[i, 'Source/Target'] = 1
    return object_dfs

## Create Homogenous PyG dataset

In [51]:
def convert_to_tensors(df_rels_filtered, combined_df):
    node_id_to_index = {node_id: index for index, node_id in enumerate(combined_df['Node ID'])}
    df_rels_filtered.loc[:, 'Source Index'] = df_rels_filtered['Source ID'].map(node_id_to_index)
    df_rels_filtered.loc[:, 'Target Index'] = df_rels_filtered['Target ID'].map(node_id_to_index)

    target_nodes_tensor = torch.from_numpy(df_rels_filtered['Target Index'].values)
    source_nodes_tensor = torch.from_numpy(df_rels_filtered['Source Index'].values)
    edge_index = torch.stack((source_nodes_tensor,target_nodes_tensor))
    df = combined_df.sort_values('Node ID')

    # Extract features
    score_tensor = torch.tensor(df['Score'].values, dtype=torch.float).unsqueeze(1)
    source_target_tensor = torch.tensor(df['Source/Target'].values, dtype=torch.float).unsqueeze(1)

    x = torch.stack([score_tensor,source_target_tensor], dim=1)
    y = torch.tensor(df['Shortest Path'].values, dtype=torch.float).unsqueeze(1)

    return df, x, y, edge_index

In [ ]:
graph = Graph("http://localhost:7474", auth=("neo4j", "bloodhoundcommunityedition"))
data_dir = ""
data = []

#Main generation function
print("===== Start =====")
for filename in os.listdir(data_dir):
    if filename.endswith(".json"):
        print("----- Starting process for new graph -----")
        print(f"Now processing: {filename} ")

        file_path = os.path.join(data_dir, filename)
        print("Clearing database")
        clear_neo4j_database(graph)

        print("Loading json file")
        load_graph_from_json(graph, file_path)

        print("Extracting features")
        object_dfs = {}
        for object_type, properties in object_types_and_properties.items():
            # object_dfs[object_type] = extract_features(graph, object_type, properties).dropna()
            imputer = SimpleImputer(strategy='most_frequent')  # Or 'median', 'most_frequent'
            object_dfs[object_type] = extract_features(graph, object_type, properties)
            for property in object_dfs[object_type].columns:
                # Take first element if it is a list
                object_dfs[object_type][property] = object_dfs[object_type][property].apply(lambda x: x[0] if isinstance(x, list) else x)
                # Set boolean to False if null
                object_dfs[object_type][property] = object_dfs[object_type][property].apply(lambda x: False if x == "null" or x is None else x) 
            object_dfs[object_type][:] = imputer.fit_transform(object_dfs[object_type])  # Impute in place
        
        print("Adding additional columns")        
        object_dfs = add_columns(object_dfs)

        print("Extracting relationships")
        relationship_df = extract_relationships(graph, relationship_types)
        relationship_df.to_csv('relationship_features.csv', index=False)

        print("Calculating and adding scores")
        object_dfs = calculate_and_add_scores(object_dfs)
        object_dfs = add_relationship_scores(object_dfs, relationship_df)

        print("Computing shortest path")
        combined_df, df_rels_filtered = create_dataframes(object_dfs, relationship_df)
        source_node, target_node, shortest_path = calculate_shortest_path(combined_df, df_rels_filtered, object_dfs)

        print("Adding shortest path to dataframes")
        object_dfs = add_shortest_path(object_dfs, shortest_path, target_node, source_node)
        combined_df, df_rels_filtered = create_dataframes(object_dfs, relationship_df)

        print("Convert to tensors and add to dataset")
        df, x, y, edge_index = convert_to_tensors(df_rels_filtered, combined_df)
        data.append(Data(x=x.squeeze(), edge_index=edge_index, y=y))

torch.save(data, "dataset_spi.pt")

## Load dataset

In [ ]:
data = torch.load("dataset_spi.pt")
print(f'Number of graphs: {len(data)}')
print(f'Number of features: {data[0].num_features}')
print(f'Number of classes: {len(torch.unique(data[0].y))}')

In [ ]:
torch.manual_seed(12345)  # For reproducibility
num_graphs = len(data)
shuffled_indices = torch.randperm(num_graphs)

# Define split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate split sizes
train_size = int(train_ratio * num_graphs)
val_size = int(val_ratio * num_graphs)
test_size = num_graphs - train_size - val_size

# Split the dataset
train_dataset = [data[i] for i in shuffled_indices[:train_size]]
val_dataset = [data[i] for i in shuffled_indices[train_size:train_size + val_size]]
test_dataset = [data[i] for i in shuffled_indices[train_size + val_size:]] 

## Model

### Original model

In [209]:
import torch
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.utils import add_self_loops


class GCNConv(MessagePassing):
    def __init__(self, in_channels, hidden_channels):
        super().__init__(aggr='add')  # "Add" aggregation
        self.lin = torch.nn.Linear(in_channels, hidden_channels)

    def forward(self, x, edge_index):
        # Step 1: Linearly transform node feature matrix
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)

        # Step 2: Propagate messages to destination nodes
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return x_j

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin_final = torch.nn.Linear(hidden_channels, 2)


    def forward(self, x, edge_index):
        # Convolutional operations
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        # Final linear transformation
        x = self.lin_final(x)
        return x

### Flexible model

In [210]:
import torch
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.utils import add_self_loops
import torch.nn as nn

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):  # Renamed hidden_channels to out_channels for clarity
        super().__init__(aggr='add')
        self.lin = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return x_j

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers, out_channels):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GCNConv(in_channels, hidden_channels))  # First layer

        for _ in range(num_layers - 2): # Hidden layers
            self.layers.append(GCNConv(hidden_channels, hidden_channels))

        self.layers.append(GCNConv(hidden_channels, out_channels))  # Last layer
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.layers):
            x = conv(x, edge_index)
            if i < len(self.layers) - 1:
                x = F.relu(x)
                x = self.dropout(x)
        return x

### Grid Search

In [ ]:
import torch
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import f1_score, classification_report

import numpy as np
import os

def plot_learning_curves(history, params, save_dir='learning_curve_macrof1'):
    """Plot and save learning curves for a parameter combination"""
    plt.figure(figsize=(12, 4))
    
    # Loss subplot
    plt.subplot(1, 2, 1)
    plt.plot(history['train_losses'], label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    
    # Accuracy subplot
    plt.subplot(1, 2, 2)
    plt.plot(history['train_accuracies'], label='Training Accuracy')
    plt.plot(history['val_accuracies'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    
    # Create title with parameters
    plt.suptitle(f'Layers: {params["num_layers"]}, Hidden: {params["hidden_channels"]}, LR: {params["learning_rate"]}')
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Save plot
    filename = f'layers_{params["num_layers"]}_hidden_{params["hidden_channels"]}_lr_{params["learning_rate"]}.png'
    plt.savefig(os.path.join(save_dir, filename), bbox_inches='tight', dpi=300)
    plt.close()

def train_model(model, train_dataset, val_dataset, learning_rate, num_epochs=50, warmup_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    weights = torch.tensor([0.00035, 0.1])
    criterion = torch.nn.CrossEntropyLoss(weight=weights)
    
    # Track best metrics (initialize with worst possible values)
    best_metrics = {
        'train_loss': float('inf'),
        'train_acc': 0,
        'val_acc': 0,
        'val_f1': 0,
        'epoch': 0,
        'is_warmup': True  # Flag to indicate if we're still in warmup period
    }
    
    # History tracking
    history = {
        'train_losses': [],
        'train_accuracies': [],
        'val_accuracies': [],
        'val_f1_scores': [],
        'crs' : []
    }
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_loss = 0
        correct_train = 0
        total_train = 0
        
        for i in range(len(train_dataset)):
            data_object = train_dataset[i]
            out = model(data_object.x, data_object.edge_index)
            optimizer.zero_grad()
            y = data_object.y.long().squeeze()
            
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            pred_train = out.argmax(dim=1)
            correct_train += (pred_train == y).sum().item()
            total_train += y.numel()
        
        # Calculate training metrics
        train_loss = epoch_loss / len(train_dataset)
        train_acc = correct_train / total_train
        
        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        val_predictions = []
        val_labels = []
        
        with torch.no_grad():
            for i in range(len(val_dataset)):
                data_object = val_dataset[i]
                out = model(data_object.x, data_object.edge_index)
                y = data_object.y.long().squeeze()
                
                pred_val = out.argmax(dim=1)
                correct_val += (pred_val == y).sum().item()
                total_val += y.numel()
                
                val_predictions.extend(pred_val.cpu().numpy())
                val_labels.extend(y.cpu().numpy())
        
        # Calculate validation metrics
        val_acc = correct_val / total_val
        val_f1 = f1_score(val_labels, val_predictions, average='macro')
        # Calculate classification report (includes precision, recall, F1-score)
        target_names = ['Not part of path', 'Part of path']  # Adjust class names as needed
        cr = classification_report(val_labels, val_predictions, target_names=target_names, zero_division=0)
        
        # Store history
        history['train_losses'].append(train_loss)
        history['train_accuracies'].append(train_acc)
        history['val_accuracies'].append(val_acc)
        history['val_f1_scores'].append(val_f1)
        history['crs'].append(cr)
        
        # Update best metrics only after warmup period
        if epoch >= warmup_epochs:
            if best_metrics['is_warmup']:
                # First update after warmup - set initial "best" values
                best_metrics = {
                    'train_loss': train_loss,
                    'train_acc': train_acc,
                    'val_acc': val_acc,
                    'val_f1': val_f1,
                    'epoch': epoch,
                    'is_warmup': False,
                    'cr': cr
                }
            else:
                # Normal updates after warmup
                if val_f1 > best_metrics['val_f1']:
                    best_metrics['train_loss'] = train_loss
                    best_metrics['train_acc'] = train_acc
                    best_metrics['val_acc'] = val_acc
                    best_metrics['val_f1'] = val_f1
                    best_metrics['epoch'] = epoch
                    best_metrics['cr'] = cr
        
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
            print(f"Val - Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
            
    return best_metrics, history

def grid_search(train_dataset, val_dataset, in_channels, out_channels, warmup_epochs=10):
    param_grid = {
        'num_layers': [1, 2, 3],
        'hidden_channels': [16, 32, 64],
        'learning_rate': [0.0001, 0.001, 0.01]
    }
    
    param_combinations = list(itertools.product(
        param_grid['num_layers'],
        param_grid['hidden_channels'],
        param_grid['learning_rate']
    ))
    
    results = []
    best_overall = {
        'train_loss': float('inf'),
        'train_acc': 0,
        'val_acc': 0,
        'val_f1': 0,
        'params': None,
        'model': None
    }
    
    total_combinations = len(param_combinations)
    
    for idx, (num_layers, hidden_channels, lr) in enumerate(param_combinations, 1):
        print(f"\nTesting combination {idx}/{total_combinations}:")
        print(f"Layers: {num_layers}, Hidden Channels: {hidden_channels}, Learning Rate: {lr}")
        
        model = GCN(in_channels, hidden_channels, num_layers, out_channels)
        best_metrics, history = train_model(
            model,
            train_dataset,
            val_dataset,
            learning_rate=lr,
            warmup_epochs=warmup_epochs
        )
        
        # Create parameter dictionary for current combination
        current_params = {
            'num_layers': num_layers,
            'hidden_channels': hidden_channels,
            'learning_rate': lr,
            'train_loss': best_metrics['train_loss'],
            'train_acc': best_metrics['train_acc'],
            'val_acc': best_metrics['val_acc'],
            'val_f1': best_metrics['val_f1'],
            'best_epoch': best_metrics['epoch'],
            'cr': best_metrics['cr']
        }
        
        # Plot and save learning curves for this combination
        plot_learning_curves(history, current_params)
        
        results.append(current_params)
        
        # Update best overall based on validation F1 score
        if current_params['val_f1'] > best_overall['val_f1']:
            best_overall['train_loss'] = current_params['train_loss']
            best_overall['train_acc'] = current_params['train_acc']
            best_overall['val_acc'] = current_params['val_acc']
            best_overall['val_f1'] = current_params['val_f1']
            best_overall['cr'] = current_params['cr']
            best_overall['params'] = current_params
            best_overall['model'] = model.state_dict()
        
        print("\nCurrent combination metrics:")
        print(f"Train Loss: {current_params['train_loss']:.4f}")
        print(f"Train Accuracy: {current_params['train_acc']:.4f}")
        print(f"Val Accuracy: {current_params['val_acc']:.4f}")
        print(f"Val F1 Score: {current_params['val_f1']:.4f}")
        print(f"Best Epoch: {current_params['best_epoch']}")
        print("\nClassification Report:")
        print(current_params['cr'])


        
        print("\nBest overall metrics so far:")
        print(f"Train Loss: {best_overall['train_loss']:.4f}")
        print(f"Train Accuracy: {best_overall['train_acc']:.4f}")
        print(f"Val Accuracy: {best_overall['val_acc']:.4f}")
        print(f"Val F1 Score: {best_overall['val_f1']:.4f}")
        print(f"Classification report: \n {best_overall['cr']}")

    return results, best_overall['params'], best_overall['model']

# Example usage
in_channels = 2
out_channels = 2

# Perform grid search with 10 epochs warmup
results, best_params, best_model = grid_search(
    train_dataset, 
    val_dataset, 
    in_channels, 
    out_channels,
    warmup_epochs=10  # Adjust this value as needed
)

# Print best parameters and metrics
print("\nBest Parameters:")
print(f"Number of Layers: {best_params['num_layers']}")
print(f"Hidden Channels: {best_params['hidden_channels']}")
print(f"Learning Rate: {best_params['learning_rate']}")
print(f"Training Loss: {best_params['train_loss']:.4f}")
print(f"Training Accuracy: {best_params['train_acc']:.4f}")
print(f"Validation Accuracy: {best_params['val_acc']:.4f}")
print(f"Validation F1 Score: {best_params['val_f1']:.4f}")
print(f"Best Epoch: {best_params['best_epoch']}")

### Training and testing

In [ ]:
import torch
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

num_layers = 1
hidden_channels = 16
learning_rate = 0.0001
num_epochs = 50

num_features = 2
num_classes = 2

model = GCN(in_channels, hidden_channels, num_layers, out_channels)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
weights = torch.tensor([0.00035, 0.1])
criterion = torch.nn.CrossEntropyLoss(weight=weights)

# Lists to store loss and accuracy values for plotting
train_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Training
    model.train()
    epoch_loss = 0
    correct_train = 0
    total_train = 0

    # For data_object in train_dataset:
    for i in range(len(train_dataset)):
        data_object = train_dataset[i]
        out = model(data_object.x, data_object.edge_index)
        optimizer.zero_grad()
        y = data_object.y.long().squeeze() 
        loss = criterion(out, y)  # Calculate loss on all nodes
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()  # Accumulate loss

        # Compare predictions
        pred_train = out.argmax(dim=1)
        correct_train += (pred_train == y).sum().item()
        total_train += y.numel()

    train_accuracy = correct_train / total_train
    train_accuracies.append(train_accuracy)
    train_losses.append(epoch_loss / len(train_dataset))  # Average loss

    # Validating
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for i in range(len(val_dataset)):
            data_object = val_dataset[i]
            out = model(data_object.x, data_object.edge_index)
            y = data_object.y.long().squeeze()

            pred_val = (out.argmax(dim=1))
            #print(f"Prediction: {pred_val} \n")
            correct_val += (pred_val == y).sum().item()
            total_val += y.numel()

    val_accuracy = correct_val / total_val
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, "
            f"Train Acc: {train_accuracy:.4f}, Validation Acc: {val_accuracy:.4f}")

# Plot the loss and accuracy curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluation
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for i in range(len(test_dataset)):
        data_object = test_dataset[i]
        out = model(data_object.x, data_object.edge_index)
        y = data_object.y.long().squeeze()

        pred_test = (out.argmax(dim=1))
        #print(f"Prediction: {pred_val} \n")
        correct_test += (pred_test == y).sum().item()
        total_test += y.numel()

test_accuracy = correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        data_object = test_dataset[i]
        out = model(data_object.x, data_object.edge_index)
        y = data_object.y.long().unsqueeze(1)

        pred_test = (out.argmax(dim=1))
        y_true.extend(y.numpy().flatten().tolist())
        y_pred.extend(pred_test.numpy().flatten().tolist())

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate classification report (includes precision, recall, F1-score)
target_names = ['Not part of path', 'Part of path']  # Adjust class names as needed
cr = classification_report(y_true, y_pred, target_names=target_names)
print("\nClassification Report:")
print(cr)

# Calculate overall accuracy (can cross-check with the report)
accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Extract TP, TN, FP, FN for manual metric calculation if needed
TN, FP, FN, TP = cm.ravel()

#Manual calculation of metrics for verification if needed
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")

### Other

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_features = dataset[0].x.shape[1]
num_classes = len(set([label.item() for data in data_list for label in data.y]))  # Assuming labels are from 0 to num_classes-1


In [104]:
model = GCN(in_channels=num_features, hidden_channels=16, out_channels=num_classes)


### New Model

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_features = dataset[0].x.shape[1]
num_classes = len(set([label.item() for data in data_list for label in data.y]))  # Assuming labels are from 0 to num_classes-1


In [159]:
model = GCN(in_channels=num_features, hidden_channels=16, out_channels=num_classes)

In [ ]:
import torch
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

#model = GCN(hidden_channels=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
weights = torch.tensor([0.001, 0.1])
criterion = torch.nn.BCELoss(weight=weights)

num_epochs = 10

# Lists to store loss and accuracy values for plotting
train_losses = []
train_accuracies = []
test_accuracies = []

for epoch in range(num_epochs):
    # Training
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    for data_object in train_loader:
        optimizer.zero_grad()
        out = model(data_object.x, data_object.edge_index)
        y = data_object.y.float()
        loss = criterion(out, target_one_hot)  # Calculate loss on all nodes
        loss.backward()
        optimizer.step()

        # Compare predictions for each class, then check if BOTH are correct
        correct_predictions = (pred == target_one_hot).all(dim=1)  
        correct += correct_predictions.sum().item()
        total += data_object.y.size(0)

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)
    train_accuracy = correct / total
    train_accuracies.append(train_accuracy)

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data_object in test_loader:
            out = model(data_object.x, data_object.edge_index)
            pred = (out > 0.5).float()  # Get binary predictions (0 or 1)

            # Convert data_object.y to one-hot for comparison with pred
            target_one_hot = data_object.y.long().squeeze(1)  

            # Compare predictions for each class, then check if BOTH are correct
            correct_predictions = (pred == target_one_hot).all(dim=1)  
            correct += correct_predictions.sum().item()
            total += data_object.y.size(0)

    test_accuracy = correct / total
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}, Test Acc: {test_accuracy:.4f}")

# Plot the loss and accuracy curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()